## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [5]:
import pandas as pd

In [7]:
df=pd.read_csv('train.csv', on_bad_lines='skip',engine='python')

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [11]:
df.shape

(19493, 5)

In [13]:
df.isnull().sum()

,0
id,0
title,522
author,1858
text,54
label,16


In [14]:
###Drop Nan Values
df=df.dropna()

In [15]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [16]:
df.shape

(17115, 5)

In [17]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [18]:
## Get the Dependent features
y=df['label']

In [19]:
X.shape

(17115, 4)

In [20]:
y.shape

(17115,)

In [21]:
import tensorflow as tf

In [22]:
tf.__version__

'2.17.1'

In [23]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [24]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [25]:
messages=X.copy()

In [26]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [27]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
19488,19473,Artificial intelligence ‘robot’ says Trump wil...,Admin,Other Writers \nAn Artificial Intelligence (AI...
19489,19474,Comment on Sweden Bans Christmas Street Lights...,jaded,"Posted on October 26, 2016 by Paul Joseph Wats..."
19490,19475,Arrests for Cannabis Possession Outnumber Arre...,The Mind Unleashed,Arrests for Cannabis Possession Outnumber Arre...
19491,19476,Clinton Vs. Trump: Latest Electoral Prediction...,beforeitsnews.com,(Before It's News)\nIt is fun to look at polls...


In [29]:
messages.reset_index(inplace=True)

In [30]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
17110,19488,19473,Artificial intelligence ‘robot’ says Trump wil...,Admin,Other Writers \nAn Artificial Intelligence (AI...
17111,19489,19474,Comment on Sweden Bans Christmas Street Lights...,jaded,"Posted on October 26, 2016 by Paul Joseph Wats..."
17112,19490,19475,Arrests for Cannabis Possession Outnumber Arre...,The Mind Unleashed,Arrests for Cannabis Possession Outnumber Arre...
17113,19491,19476,Clinton Vs. Trump: Latest Electoral Prediction...,beforeitsnews.com,(Before It's News)\nIt is fun to look at polls...


In [31]:
import nltk
import re
from nltk.corpus import stopwords

In [32]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [33]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [34]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [35]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [36]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2579, 4940, 177, 948, 1758, 3623, 1525, 4423, 2490, 21],
 [3946, 435, 3320, 4902, 308, 2300, 2391],
 [2612, 2739, 199, 1221],
 [1093, 3689, 3986, 3477, 547, 4403],
 [3909, 308, 3533, 2086, 1843, 2504, 308, 4326, 1839, 1043],
 [1228,
  4947,
  4107,
  1673,
  3564,
  2154,
  3949,
  1341,
  4226,
  1917,
  770,
  665,
  3548,
  606,
  2391],
 [1986, 4882, 2839, 4276, 3692, 4907, 3344, 1759, 3463, 4161, 2235],
 [3747, 1149, 3797, 3022, 2152, 4425, 2154, 4871, 3463, 4161, 2235],
 [4074, 4888, 433, 1049, 1965, 1249, 4512, 1271, 2154, 3818],
 [1556, 3075, 4375, 2237, 618, 1294, 2863, 449],
 [290, 2941, 3654, 3684, 2916, 3691, 3795, 1810, 1995, 137, 2964],
 [3477, 1847, 1758, 1249, 2154, 2152],
 [1805, 4511, 3642, 3253, 3651, 1614, 4338, 3420, 1455],
 [3229, 2826, 1960, 2604, 2301, 1327, 2506, 3463, 4161, 2235],
 [1148, 3655, 1025, 3083, 3296, 3463, 4161, 2235],
 [2568, 2372, 2494, 2941, 2272, 3197, 4793, 673, 2060, 1306],
 [3662, 20, 435],
 [1622, 2291, 2904, 498, 2154, 4903, 3690, 2391],

In [37]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [38]:
onehot_repr[1]

[3946, 435, 3320, 4902, 308, 2300, 2391]

### Embedding Representation

In [39]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2579 4940  177 ...    0    0    0]
 [3946  435 3320 ...    0    0    0]
 [2612 2739  199 ...    0    0    0]
 ...
 [4793  468  301 ...    0    0    0]
 [3320 3817 2154 ...    0    0    0]
 [2349 1249 4571 ...    0    0    0]]


In [40]:
embedded_docs[1]

array([3946,  435, 3320, 4902,  308, 2300, 2391,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [41]:
embedded_docs[0]

array([2579, 4940,  177,  948, 1758, 3623, 1525, 4423, 2490,   21,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [61]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [62]:
len(embedded_docs),y.shape

(17115, (17115,))

In [63]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [64]:
X_final.shape,y_final.shape

((17115, 20), (17115,))

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [66]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7498 - loss: 0.4423 - val_accuracy: 0.9237 - val_loss: 0.1829
Epoch 2/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9525 - loss: 0.1326 - val_accuracy: 0.9240 - val_loss: 0.1848
Epoch 3/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9729 - loss: 0.0919 - val_accuracy: 0.9145 - val_loss: 0.2506
Epoch 4/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9777 - loss: 0.0691 - val_accuracy: 0.9047 - val_loss: 0.2632
Epoch 5/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9830 - loss: 0.0580 - val_accuracy: 0.9171 - val_loss: 0.2790
Epoch 6/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9881 - loss: 0.0369 - val_accuracy: 0.9163 - val_loss: 0.2984
Epoch 7/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9905 - loss: 0.0307 - val_accuracy: 0.9109 - val_loss: 0.3361
Epoch 8/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9917 - loss: 0.0296 - val_accuracy: 0.

### Adding Dropout

### Performance Metrics And Accuracy

In [67]:
y_pred=model.predict(X_test)

177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [68]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [69]:
from sklearn.metrics import confusion_matrix

In [70]:
confusion_matrix(y_test,y_pred)

array([[3004,  206],
       [ 397, 2041]])

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8932365439093485

In [60]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.57      1.00      0.72      3210
         1.0       0.00      0.00      0.00      2438

    accuracy                           0.57      5648
   macro avg       0.28      0.50      0.36      5648
weighted avg       0.32      0.57      0.41      5648



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
